# Import libraries 

In [ ]:
import numpy
from sklearn.metrics import f1_score, precision_score, recall_score
numpy.float = float
numpy.int = numpy.int_
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from numpy import array
from scipy import ndimage as ndi
from tqdm.notebook import tqdm
import neuclid as nc
from volume import Volume
from scipy import ndimage
import os
import keras
import tensorflow as tf
from tensorflow.python.client import device_lib
import logging
import matplotlib.ticker as ticker
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [ ]:
"""load data generator for trainig and testing data"""
import Data_generator


In [ ]:
%config InlineBackend.figure_format='retina' # to make the plot more resolution
plt.style.use("default")

In [ ]:
#enable GPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
logging.getLogger('tensorflow').setLevel(logging.ERROR)
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(False)
os.environ["KERAS_BACKEND"] = "tensorflow"
tf.debugging.set_log_device_placement(False)
sess = tf.compat.v1.Session()
print(device_lib.list_local_devices())
print(tf.config.list_physical_devices("GPU"))
# test if GPU is enabled on local device: macOS m1 mps, Windows GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))
print("Device Name:", tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))
tf.config.list_physical_devices("GPU")

# Load volumes

In [53]:
nrrd_volumes, cochlea_l, cochlea_r = Data_generator.volume_ct_load(Data_generator.base_dir) # here to take the loaded volumes ground truch data, with the default author provided volume data

  0%|          | 0/1 [00:00<?, ?it/s]

0002_toload
testvolume/0002_toload/0002.nrrd
	leftpath.parent.name[:4] is 0002
path  is testvolume/0002_toload/0002.nrrd
	good segmentation: GT
cochlea_fcsv_files is [PosixPath('testvolume/0002_toload/left/GT/Cochlea.fcsv')]
	 -> ['Cochlea.fcsv']

	rightpath.parent.name[:4] is 0002
path  is testvolume/0002_toload/0002.nrrd
	good segmentation: GT
cochlea_fcsv_files is [PosixPath('testvolume/0002_toload/right/GT/Cochlea_r.fcsv')]
	 -> ['Cochlea_r.fcsv']



# Start the training on the fly (Optional: activate it when a local model needs to be trained) 

In [ ]:
"""training function to train from scratch the SMICNet locally, defaut set to False """
train_activate=False  # set to True to train the model, this will take a long duration to train 
if train_activate:
    import Training 

# Model application/inference

### load different model trained with different quantity volumes

In [ ]:
model_SW_4classes_trainedwith1volume = keras.models.load_model(
    "TrainedNetworkWeights/model_epoch_100_1volume_authorprovided.h5"
)

In [ ]:
model_SW_4classes_trainedwith3volumes = keras.models.load_model(
    "TrainedNetworkWeights/model_epoch_100_3volumes_authorprovided.h5"
)

In [ ]:
model_SW_4classes_trainedwith5volumes = keras.models.load_model(
    "TrainedNetworkWeights/model_epoch_100_5volumes_authorprovided.h5"
)

In [ ]:
model_SW_4classes_trainedwith7volumes = keras.models.load_model(
    "TrainedNetworkWeights/model_epoch_100_7volumes_authorprovided.h5"
)

In [ ]:
model_SW_4classes_trainedwith9volumes = keras.models.load_model(
    "TrainedNetworkWeights/model_epoch_100_9volumes_authorprovided.h5"
)

In [ ]:
model_SW_4classes_trainedwith11volumes = keras.models.load_model(
    "TrainedNetworkWeights/model_epoch_100_11volumes_authorprovided.h5"
)

## classification performance

### Evaluation of images from generator

In [ ]:
nrrd_volumes.keys()

In [ ]:
test_data_gen11 = Data_generator.Data_augmentation_Generator(
    batch_size=32,
    selected_volumes=['0002'], # train_ids, choose based on the nrrd_volumes.keys()
    steps1=4,
    steps2=4,
    steps3=4,#to adjust how many 2d data to generate to test
    alpha_range=10,
    beta_range=10,
    gamma_range=330,
    window_size=81,
    gradual_rotated_padding_size=3,
    rotated_augmentation_size=6
)

In [ ]:
x, y = next(iter(test_data_gen11)) 

In [ ]:
apex_SW12 = model_SW_4classes_trainedwith5volumes.predict(x)
apex_SW12  # epoch 12

In [ ]:
y  # to compare the ground truth label of this batch

### quick look which class the loaded 2D sub image belongs to 

In [ ]:
def which_class(label):
    if label == 0:
        return ("This is class cochlea apex! ")
    elif label == 1:
        return ("This is class cochlea basal!")
    elif label == 2:
        return ("This is class cochlea round window!")
    elif label == 3:
        return ("This is class background!")
    else: 
        print("Invalid class,there is an error")

In [ ]:
def display_image_and_class(i):
    input_2dimg=tf.expand_dims(x[i], axis=0)
    predicted_2dlabel=model_SW_4classes_trainedwith5volumes.predict(input_2dimg)

    plt.imshow(x[i].squeeze(), cmap='gray')
    plt.show()
    print("Ground truth class:", which_class(np.argmax(y[i])))
    print("Predicted class:", which_class(np.argmax(predicted_2dlabel)))
    if np.argmax(y[i])== np.argmax(predicted_2dlabel):
        print("The SMICNet model is right on this given image's class!!")
    else:
        print("The SMICNet model made a mistake on this given image's class!!")

In [ ]:
display_image_and_class(1) # here can change the int value to switch the image to test, here shows the class of the center of the image loaded

### Optional: CNN layers visualization

In [ ]:
layyer_visualisation=False # optional: change to True to see each layer's features extraction

In [ ]:
if layyer_visualisation:
    x_input = tf.expand_dims(x[0], axis=0)
    layer_outputs = []
    for layer in model_SW_4classes_trainedwith5volumes.layers:
        layer_model = tf.keras.Model(
            inputs=model_SW_4classes_trainedwith5volumes.input, outputs=layer.output
        )
        output = layer_model.predict(x_input)
        layer_outputs.append(output)


    n_layers = len(layer_outputs)
    n_cols = 8
    n_rows = int(np.ceil(n_layers / n_cols))

    fig, axs = plt.subplots(n_rows, n_cols, figsize=(2 * n_cols, 2 * n_rows))
    axs = axs.flatten()

    for i, ax in enumerate(axs):
        if i < n_layers:
            output = layer_outputs[i]

            if output.ndim == 4:
                ax.imshow(output[0, :, :, 0])
            elif output.ndim == 2:
                ax.imshow(output, cmap="viridis")
            else:
                ax.text(
                    0.5,
                    0.5,
                    f"Cannot display\noutput of shape\n{output.shape}",
                    horizontalalignment="center",
                    verticalalignment="center",
                    fontsize=12,
                )

            ax.axis("off")
            ax.set_title(f"Layer {i + 1}")
        else:
            ax.axis("off")
    plt.savefig(
        f"layervisualisation_batch1images.png",
        format="png",
        dpi=400,
        bbox_inches="tight",
    )


    plt.show()

In [ ]:
def classification_score_formatready(
    set_toselect,
    quantity_nrrdstochoose,
    steps1,
    steps2,
    steps3,
    model,
    model_indice,
    batch_size=32,
):
    df_scores_list = list()
    f1_scores_per_class_list = list()
    f1_macro_list = list()
    f1_imbalanced_differentclass_class_list = list()
    f1_weighted_list = list()
    prob_scores_list2_array_list = list()
    y_pred_labelrray_list = list()
    y_true_list2_array_list = list()
    data_list = list()
    for i in range(len(set_toselect) - quantity_nrrdstochoose):
        selected_volumes = [set_toselect[i]]
        # print(selected_volumes)
        y_true_list = list()
        prob_scores_list = list()
        y_true_list2 = list()
        y_pred_label = list()
        prob_scores_list2 = list()

        batch_size = batch_size

        test_data_gen11 = Data_generator.Data_augmentation_Generator(
            batch_size=batch_size,
            selected_volumes=selected_volumes,  # train_ids
            steps1=steps1,
            steps2=steps2,
            steps3=steps3,
            alpha_range=10,
            beta_range=10,
            gamma_range=330,
            window_size=81,
            gradual_rotated_padding_size=17,
            rotated_augmentation_size=10,
        )
        for row in range(len(test_data_gen11)):
            img_, y_true = next(iter(test_data_gen11))
            prob_scores = model.predict(img_)
            y_true_list.append(y_true)
            prob_scores_list.append(prob_scores)

        for i in y_true_list:
            for j in range(batch_size):
                y_true_list2.append(np.argmax(i[j]))

        y_true_list2_array = np.array(y_true_list2)
        for i in prob_scores_list:
            for j in range(batch_size):
                prob_scores_list2.append(i[j])
                y_pred_label.append(i[j])

        prob_scores_list2_array = np.array(prob_scores_list2)
        y_true_list2_array_list.append(y_true_list2_array)

        y_pred_labelrray = np.array(y_pred_label)
        y_pred_label = np.argmax(y_pred_labelrray, axis=1)

        prob_scores_list2_array_list.append(prob_scores_list2_array)
        y_pred_labelrray_list.append(y_pred_labelrray)

        f1_scores_per_class = f1_score(y_true_list2_array, y_pred_label, average=None)
        precision_per_class = precision_score(
            y_true_list2_array, y_pred_label, average=None
        )
        recall_per_class = recall_score(y_true_list2_array, y_pred_label, average=None)

        f1_macro = f1_score(y_true_list2_array, y_pred_label, average="macro")
        f1_imbalanced_differentclass = f1_score(
            y_true_list2_array, y_pred_label, average="micro"
        )
        f1_weighted = f1_score(y_true_list2_array, y_pred_label, average="weighted")

        f1_scores_per_class_list.append([selected_volumes] + [f1_scores_per_class])
        f1_macro_list.append([selected_volumes] + [f1_macro])
        f1_imbalanced_differentclass_class_list.append(
            [selected_volumes] + [f1_imbalanced_differentclass]
        )
        f1_weighted_list.append([selected_volumes] + [f1_weighted])

        print(f"f1_scores_per_class is {f1_scores_per_class}")

        for i, f1_per_class in enumerate(f1_scores_per_class):
            if i == 0:
                class_label = "Apex"
            elif i == 1:
                class_label = "Basal"
            elif i == 2:
                class_label = "Round window"
            else:
                class_label = "Background"
            data_row = {
                "selected Volume": selected_volumes,
                "Class": class_label,
                "Precision per Class": precision_per_class[i],
                "Recall per Class": recall_per_class[i],
                "F1 per Class": f1_per_class,
                "F1 Macro": f1_macro,
                "F1 Micro": f1_imbalanced_differentclass,
                "F1 Weighted": f1_weighted,
            }
            data_list.append(data_row)
        df_scores = pd.DataFrame(data_list)

        df_scores.to_csv(
            f"f1_scores_SMICNet_0{model_indice}.csv",
            index=False,
        )
    return (
        df_scores,
        y_true_list2_array_list,
        f1_scores_per_class_list,
        f1_macro_list,
        f1_imbalanced_differentclass_class_list,
        f1_weighted_list,
        prob_scores_list2_array_list,
        y_pred_labelrray_list,
    )

In [ ]:
## choose the volume data index to test 
set_toselect = [
    "0002",
]

### Optional: F1 score generation based on 2D images, with different model

In [ ]:
#model trained with 1 volume
(
    df_scores_list,
    y_true_list2_array_list,
    f1_scores_per_class_list,
    f1_macro_list,
    f1_imbalanced_differentclass_class_list,
    f1_weighted_list,
    prob_scores_list2_array_list,
    y_pred_labelrray_list,
) = classification_score_formatready(
    set_toselect,
    quantity_nrrdstochoose=0,
    steps1=4,
    steps2=4,
    steps3=4,  # can change here to control the quantity of data used for the f1 score calculation
    model=model_SW_4classes_trainedwith1volume,
    model_indice=1,  # indice of trained volume's quantity
    batch_size=4,
)

In [ ]:
#model trained with 3 volumes
(
    df_scores_list,
    y_true_list2_array_list,
    f1_scores_per_class_list,
    f1_macro_list,
    f1_imbalanced_differentclass_class_list,
    f1_weighted_list,
    prob_scores_list2_array_list,
    y_pred_labelrray_list,
) = classification_score_formatready(
    set_toselect,
    quantity_nrrdstochoose=0,
    steps1=4,
    steps2=4,
    steps3=4,
    model=model_SW_4classes_trainedwith3volumes,
    model_indice=3,  # indice of trained volume's quantity
    batch_size=4,
)

In [ ]:
#model trained with 5 volumes
(
    df_scores_list,
    y_true_list2_array_list,
    f1_scores_per_class_list,
    f1_macro_list,
    f1_imbalanced_differentclass_class_list,
    f1_weighted_list,
    prob_scores_list2_array_list,
    y_pred_labelrray_list,
) = classification_score_formatready(
    set_toselect,
    quantity_nrrdstochoose=0,
    steps1=4,
    steps2=4,
    steps3=4,
    model=model_SW_4classes_trainedwith5volumes,
    model_indice=5,
    batch_size=4,
)

In [ ]:
#model trained with 7 volumes
(
    df_scores_list,
    y_true_list2_array_list,
    f1_scores_per_class_list,
    f1_macro_list,
    f1_imbalanced_differentclass_class_list,
    f1_weighted_list,
    prob_scores_list2_array_list,
    y_pred_labelrray_list,
) = classification_score_formatready(
    set_toselect,
    quantity_nrrdstochoose=0,
    steps1=4,
    steps2=4,
    steps3=4,
    model=model_SW_4classes_trainedwith7volumes,
    model_indice=7,
    batch_size=4,
)

In [ ]:
#model trained with 9 volumes
(
    df_scores_list,
    y_true_list2_array_list,
    f1_scores_per_class_list,
    f1_macro_list,
    f1_imbalanced_differentclass_class_list,
    f1_weighted_list,
    prob_scores_list2_array_list,
    y_pred_labelrray_list,
) = classification_score_formatready(
    set_toselect,
    quantity_nrrdstochoose=0,
    steps1=4,
    steps2=4,
    steps3=4,
    model=model_SW_4classes_trainedwith9volumes,
    model_indice=9,
    batch_size=4,
)

In [ ]:
#model trained with 11 volumes
(
    df_scores_list,
    y_true_list2_array_list,
    f1_scores_per_class_list,
    f1_macro_list,
    f1_imbalanced_differentclass_class_list,
    f1_weighted_list,
    prob_scores_list2_array_list,
    y_pred_labelrray_list,
) = classification_score_formatready(
    set_toselect,
    quantity_nrrdstochoose=0,
    steps1=4,
    steps2=4,
    steps3=4,
    model=model_SW_4classes_trainedwith11volumes,
    model_indice=11,
    batch_size=4,
)

###  Optional: load author provided F1 score CSV, to compare classification performance among different networks 



In [ ]:
weightsF1_score=False # optional: change to True to see the F1 score of different model trained with different volumes

In [ ]:
if weightsF1_score:
    # Load the CSV file on model trained with 1 volume
    # df_f1scores = pd.read_csv(
    #     "f1_scores_SMICNet_01_authorprovided.csv", on_bad_lines="skip"
    # )
    # # Load the CSV file on model trained with 5 volumes
    df_f1scores = pd.read_csv(
        "f1_scores_SMICNet_05_authorprovided.csv", on_bad_lines="skip"
    )
    # # Load the CSV file on model trained with 11 volumes
    # df_f1scores = pd.read_csv(
    #     "f1_scores_SMICNet_011_authorprovided.csv", on_bad_lines="skip"
    # )
else:
    df_f1scores = None
df_f1scores

## Distance: run on the 3D volume 

### Sliding Window process

In [ ]:
def tfimage_SW(img):
    window = tf.keras.preprocessing.image.array_to_img(img)
    window = tf.keras.preprocessing.image.img_to_array(window)
    window_input = tf.expand_dims(
        window, axis=0
    )  
    window_input = window_input / 255.0
    return window_input

In [ ]:
def sliding_range_creating(range_set=5):
    
    """
    to choose 
    """
    x, y, z = (
        np.arange(-range_set, range_set),
        np.arange(-range_set, range_set),
        np.arange(-range_set, range_set),
    )
    xx, yy, zz = np.meshgrid(x, y, z)
    xy_coords_r_2darray = np.column_stack(
        (xx.ravel(), yy.ravel(), zz.ravel())
    )  
    return xy_coords_r_2darray  

In [ ]:
def SW_volume_loop_sagittal_multiclass_coarse(
    reference,
    v,
    SW_model,
    corse_step=3,
    fine_step=1,
    search_range=3,
    range_set=5,
    coarse_thresh=0.92,
):
    """
    sliding proceess: 
    
    - do a first coarse sliding:  per 3 pixel, if find interesting pixels (coarse_thresh),
    do a search per 1 pixel for a range of 6 pixel (with pixel of interest in the middle)

    this function is to slide following the saggital direction, can also change it to coronal or axial direction

    """

    xy_coords_2darray = sliding_range_creating(range_set=range_set)
    out_origin_class1 = np.zeros_like(v.data, dtype="float")
    out_origin_class2 = np.zeros_like(v.data, dtype="float")
    out_origin_class3 = np.zeros_like(v.data, dtype="float")

    coordinates_correspond_list_SW = []
    global_range = xy_coords_2darray + reference
    selstected_range = global_range[::corse_step]
    for index, element in enumerate(selstected_range):
        x1, y1, z1 = element
        window_see = v.data[
            round(x1),
            round(y1) - 40 : round(y1) + 41,
            round(z1) - 40 : round(z1) + 41,
        ]
        window_see = tf.expand_dims(window_see, axis=2)

        window_input = tfimage_SW(window_see)
        SW_proba = SW_model.predict(window_input)
        if SW_proba[0][3] > coarse_thresh:
            print("background!")
        else:
            start = max(0, index * corse_step - search_range)
            end = min(len(global_range), index * corse_step + search_range + 1)
            print(f"global_range[start:end] is {global_range[start:end]}")

            for x2, y2, z2 in tqdm(global_range[start:end]):
                window_see2 = v.data[
                    round(x2),
                    round(y2) - 40 : round(y2) + 41,
                    round(z2) - 40 : round(z2) + 41,
                ]
                window_see2 = tf.expand_dims(window_see2, axis=2)

                window_input2 = tfimage_SW(window_see2)

                SW_proba2 = SW_model.predict(window_input2)
                class_label = np.argmax(SW_proba2, axis=1)
                coordinates_correspond = [x2, y2, z2]
                coordinates_correspond_list_SW.append(
                    [class_label] + [SW_proba2] + [coordinates_correspond]
                )

                if class_label == array([0]):
                    out_origin_class1[round(x2), round(y2), round(z2)] = np.max(
                        SW_proba2
                    )
                elif class_label == array([1]):
                    out_origin_class2[round(x2), round(y2), round(z2)] = np.max(
                        SW_proba2
                    )
                elif class_label == array([2]):
                    out_origin_class3[round(x2), round(y2), round(z2)] = np.max(
                        SW_proba2
                    )
                else:
                    print("background! step2")

    return (
        coordinates_correspond_list_SW,
        out_origin_class1,
        out_origin_class2,
        out_origin_class3,
    )

In [ ]:
def map_generation(
    v,
    cochlea_center_l,
    cochlea_center_l_apex,
    cochlea_center_l_rw,
    selected_volumes,
    left_ijk,
    left_ijk_apex,
    left_ijk_rw,
    cochlea_center_r,
    cochlea_center_r_apex,
    cochlea_center_r_rw,
    right_ijk,
    right_ijk_apex,
    right_ijk_rw,
    SW_model,
    model_indice=0,
    range_set=12,
):
    """for each trained weight,generate different probability map for comparasion"""
    out_origin_class1_saggitale_3volumes_left = ([],)
    out_origin_class2_saggitale_3volumes_left = ([],)
    out_origin_class3_saggitale_3volumes_left = ([],)
    initial_pointleft = (left_ijk + left_ijk_apex + left_ijk_rw) / 3
    initial_pointright = (right_ijk + right_ijk_apex + right_ijk_rw) / 3 # can do the same for the right side 

    """"LR sagittal"""

    (
        coordinates_correspond_list_SW_saggitale_3volumes_left,
        out_origin_class1_saggitale_3volumes_left,
        out_origin_class2_saggitale_3volumes_left,
        out_origin_class3_saggitale_3volumes_left,
    ) = SW_volume_loop_sagittal_multiclass_coarse(
        initial_pointleft,
        v,
        range_set=range_set,
        SW_model=SW_model,
        coarse_thresh=0.5,
    )

    np.save(
        f"m{model_indice}_probabilitymap_class1_{selected_volumes}_saggital.npy",
        out_origin_class1_saggitale_3volumes_left,
    )  # save
    np.save(
        f"m{model_indice}_probabilitymap_class2_{selected_volumes}_saggital.npy",
        out_origin_class2_saggitale_3volumes_left,
    )  # save
    np.save(
        f"m{model_indice}_probabilitymap_class3_{selected_volumes}_saggital.npy",
        out_origin_class3_saggitale_3volumes_left,
    )

In [ ]:
def get_landmark_locations_SW_centermass(
    volume=None, probability_map=None, threshold=0.9
):
    """
    get center mass: take the centeroid position

    24/05/23
    """
    labels, n_labels = ndimage.label(probability_map > threshold)
    prob_map = probability_map.copy()
    prob_map[prob_map < threshold] = 0
    print(f"N = {n_labels} landmarks found.")  
    islands_sizes = []
    for n in range(
        1, n_labels + 1
    ):  # bigger center mass will be chosen if there are 2 island(n_lables)

        size_of_island = len(labels[labels == n])
        islands_sizes.append([size_of_island, n])
    print(f"islands_sizes {islands_sizes}")
    if len(islands_sizes) == 0:
        return None
    largest_island_size, largest_island_label = list(sorted(islands_sizes))[-1]
    prob_map[
        labels != largest_island_label
    ] = 0  # we set the prob_map to zero everywhere except where the largest island is
    center_of_mass = ndi.center_of_mass(prob_map)

    print(f"landmark as center of mass in ijk = {center_of_mass}")
    if volume:
        landmark = volume.to_world(
            center_of_mass
        ) #ras coordinates

    else:
        landmark = center_of_mass

    return landmark, largest_island_size

In [ ]:
def distance_mm(v, landmarks_coordinates, ground_truth_coordinates):

    # convert ijk distance to distance in mm
    predicted_point = nc.Point3(v.to_world(list(landmarks_coordinates)))
    return np.linalg.norm(predicted_point - np.array(ground_truth_coordinates))

In [ ]:
(
    v_loaded,
    cochlea_center_l_loaded,
    cochlea_center_l_loaded_apex,
    cochlea_center_l_loaded_rw,
    left_ijk_loaded,
    left_ijk_loaded_apex,
    left_ijk_loaded_rw,
    cochlea_center_r_loaded,
    cochlea_center_r_loaded_apex,
    cochlea_center_r_loaded_rw,
    right_ijk_loaded,
    right_ijk_loaded_apex,
    right_ijk_loaded_rw,
) = Data_generator.load_volume_ijk_GT("0002")

In [ ]:
initial_pointleft_loaded = (
    left_ijk_loaded + left_ijk_loaded_apex + left_ijk_loaded_rw
) / 3  # center of these 3 landmarks, consider as intial manual input
initial_pointleft_loaded

### probability map generateration: with ex model trained with 5 volumes


In [ ]:
# genereate maps with different loaded volume
set_toselect = ["0002"]
quantity_nrrdstochoose=0 # can set the range of how many volumes' probability map to do 
for i in range(len(set_toselect) - quantity_nrrdstochoose):
    selected_volumes = set_toselect[i]
    map_generation(
        v=v_loaded,
        cochlea_center_l=cochlea_center_l_loaded,
        cochlea_center_l_apex=cochlea_center_l_loaded_apex,
        cochlea_center_l_rw=cochlea_center_l_loaded_rw,
        selected_volumes=selected_volumes,
        left_ijk=left_ijk_loaded,
        left_ijk_apex=left_ijk_loaded_apex,
        left_ijk_rw=left_ijk_loaded_rw,
        cochlea_center_r=cochlea_center_r_loaded,
        cochlea_center_r_apex=cochlea_center_r_loaded_apex,
        cochlea_center_r_rw=cochlea_center_r_loaded_rw,
        right_ijk=right_ijk_loaded,
        right_ijk_apex=right_ijk_loaded_apex,
        right_ijk_rw=right_ijk_loaded_rw,
        SW_model=model_SW_4classes_trainedwith5volumes,#switch trained model
        model_indice=5,#switch which model
        range_set=12, # here can adjust the range(voxel) of interest, lower the value, quicker the inference, but may skip the actual fiducial point
    )

### load trained probability map for calcuate the landamrk coordinates: for 3 classes


In [ ]:
probabilitymap_volume_class1 = np.load(
    "m5_probabilitymap_class1_0002_saggital.npy"
)
probabilitymap_volume_class2 = np.load(
    "m5_probabilitymap_class2_0002_saggital.npy"
)
probabilitymap_volume_class3 = np.load(
    "m5_probabilitymap_class3_0002_saggital.npy"
)

In [ ]:
probabilitymap_volume_class3.max()

### predicted landmark localization 

In [ ]:
#apex##
landmarks_predicted_class1 = get_landmark_locations_SW_centermass(
    probability_map=probabilitymap_volume_class1,
    threshold=0.9999,
)
print(
    f" predicted coordinates is {landmarks_predicted_class1} and GT coordinate is {left_ijk_loaded_apex}"
)


##basal##
# landmarks_predicted_class2 = get_landmark_locations_SW_centermass(
#     probability_map=probabilitymap_volume_class2,
#     threshold=0.99,
# )

# print(f" predicted coordinates is {landmarks_predicted_class2} and GT coordinate is {left_ijk_loaded}")

##rw####
# landmarks_predicted_class3 = get_landmark_locations_SW_centermass(
#     probability_map=probabilitymap_volume_class3,
#     threshold=0.5,
# )
# print(f" predicted coordinates is {landmarks_predicted_class3} and GT coordinate is {left_ijk_loaded_rw}")

In [ ]:
landmarks_predicted_class1[0], left_ijk_loaded_apex#apex 
# landmarks_predicted_class2[0], left_ijk_loaded#basal
# landmarks_predicted_class3[0], left_ijk_loaded_rw#round window

### distance calculation: with 0.3mm image spacing

In [ ]:
distance_apex = distance_mm(
    v_loaded, landmarks_predicted_class1[0], cochlea_center_l_loaded_apex
)
distance_apex  # mm
##can also calcuate the distance on other landmarks prediction
# distance_basal = distance_mm(v_loaded, landmarks_predicted_class2[0], cochlea_center_l_loaded)
# distance_basal
# distance_rw = distance_mm(v_loaded, landmarks_predicted_class3[0], cochlea_center_l_loaded_rw)
# distance_rw

### visualisation: compare with the ground truth landmark

Axial view

In [ ]:
## entire image 
slice_=np.int(left_ijk_loaded_apex[2])
img = v_loaded[:,:,slice_]

img = img.data.squeeze()
plt.imshow(img, cmap='gray')
predicted=landmarks_predicted_class1[0][:2]
plt.scatter(predicted[1], predicted[0], color='g', label='Predicted',s=5)   
gt=left_ijk_loaded_apex[:2]
plt.scatter(gt[1], gt[0], color='r', label='Ground Truth',s=5)  
legend = plt.legend(loc='upper right', fontsize='small', frameon=True, framealpha=1, facecolor='white', edgecolor='black')
legend.get_frame().set_linewidth(0.5)
plt.savefig('prediection_model11_apex_axial_entireimageview.png')
plt.show()

In [ ]:
## zoomed view
img = v_loaded[:,:,slice_]
img = img.data.squeeze()
plt.imshow(img, cmap='gray')
x = landmarks_predicted_class1[0][0]
y= landmarks_predicted_class1[0][1]
z= landmarks_predicted_class1[0][2]
predicted = (x, y)
plt.scatter(predicted[1], predicted[0], color='g', label='Predicted',s=10)   
x_ = left_ijk_loaded_apex[0]
y_= left_ijk_loaded_apex[1]
z_= left_ijk_loaded_apex[2]
gt=(x_, y_)
plt.scatter(gt[1], gt[0], color='r', label='Ground Truth',s=10)  
plt.xlim(predicted[1]-50, predicted[1]+50)# here to zoom if want to take closer look 
plt.ylim(predicted[0]-50, predicted[0]+50)
legend = plt.legend(loc='upper right', fontsize='small', frameon=True, framealpha=1, facecolor='white', edgecolor='black')
legend.get_frame().set_linewidth(0.5)
plt.savefig('prediction_model11_apex_axial.png')
plt.show()

Coronal view

In [ ]:
slice_=np.int(left_ijk_loaded_apex[1])
img = v_loaded[:,slice_,:]
img = img.data.squeeze()
plt.imshow(img, cmap='gray')
x = landmarks_predicted_class1[0][0]
y= landmarks_predicted_class1[0][1]
z= landmarks_predicted_class1[0][2]
predicted = (x, z)
plt.scatter(predicted[1], predicted[0], color='g', label='Predicted',s=10)   
x_ = left_ijk_loaded_apex[0]
y_= left_ijk_loaded_apex[1]
z_= left_ijk_loaded_apex[2]
gt=(x_, z_)
plt.scatter(gt[1], gt[0], color='r', label='Ground Truth',s=10)  
plt.xlim(predicted[1]-50, predicted[1]+50)# here to zoom if want to take closer look 
plt.ylim(predicted[0]-50, predicted[0]+50)
legend = plt.legend(loc='upper right', fontsize='small', frameon=True, framealpha=1, facecolor='white', edgecolor='black')
legend.get_frame().set_linewidth(0.5)
plt.savefig('prediction_model11_apex_coronal.png')
plt.show()

Sagittal view

In [ ]:
slice_=np.int(left_ijk_loaded_apex[0])
img = v_loaded[slice_,:,:]
img = img.data.squeeze()
plt.imshow(img, cmap='gray')
x = landmarks_predicted_class1[0][0]
y= landmarks_predicted_class1[0][1]
z= landmarks_predicted_class1[0][2]
predicted = (y, z)
plt.scatter(predicted[1], predicted[0], color='g', label='Predicted',s=10)   
x_ = left_ijk_loaded_apex[0]
y_= left_ijk_loaded_apex[1]
z_= left_ijk_loaded_apex[2]
gt=(y_, z_)
plt.scatter(gt[1], gt[0], color='r', label='Ground Truth',s=10)  
plt.xlim(predicted[1]-50, predicted[1]+50)# here to zoom if want to take closer look 
plt.ylim(predicted[0]-50, predicted[0]+50)
legend = plt.legend(loc='upper right', fontsize='small', frameon=True, framealpha=1, facecolor='white', edgecolor='black')
legend.get_frame().set_linewidth(0.5)
plt.savefig('prediction_model11_apex_saggital.png')
plt.show()